#### 인터페이스가 간단하면 클래스 대신 함수를 받자!!!

파이썬은 일급 함수를 갖추었기 때문에 함수를 매개변수로 전달하고 참조할 수 있다.

예를들어 defaultdict 클래스를 사용자화 한다고 하면


In [7]:
from collections import defaultdict

def log_missing():
    print('key added')
    return 0

current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9),
]

result = defaultdict(log_missing, current)
print('Before: ', dict(result))
for key, amount in increments:
    result[key] += amount
print('After: ', dict(result))

Before:  {'green': 12, 'blue': 3}
key added
key added
After:  {'green': 12, 'blue': 20, 'red': 5, 'orange': 9}


_log_missing_ 같은 함수를 넘기면 결정 동작과 부작용을 분리하므로 API를 쉽게 구축하고 테스트할 수 있다.
예를 들어 기본값 후크를 defaultdict에 넘겨서 찾을 수 없는 키의 총 개수를 센다고 해보자.

첫번째 방법은 상태 보존 클로저를 사용하는 것 이다.

In [10]:
def increment_with_report(current, increments):
    added_count = 0
    
    def missing():
        nonlocal added_count # 상태 보존 클로저
        added_count += 1
        return 0
    
    result = defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount
        
    return result, added_count

In [13]:
result, count = increment_with_report(current, increments)
assert count == 2

nonlocal 키워드를 사용해서 클로저를 정의하여 문제를 해결하는것 보다 더 좋은 방법은
보존할 상태를 캡슐화하는 작은 클래스를 정의하는 것이다.

In [14]:
class CountMissing(object):
    def __init__(self):
        self.added = 0
    
    def missing(self):
        self.added += 1
        return 0

다른 언어에서 라면 CountMissing의 인터페이스를 수용하도록 defaultdict를 수정해야 한다고 생각하겠지만,
파이썬에서는 일급 함수 덕분에 객체로 CountMissing.missing 메서드를 직접 참조해서
defaultdict의 기본값 후크로 넘길 수 있다.

In [16]:
counter = CountMissing()
result = defaultdict(counter.missing, current)

for key, amount in increments:
    result[key] += amount
assert counter.added == 2

_increment_with_report_ 함수를 사용한 방법보다 명확하지만,
defaultdict 와 연계해서 사용한 예를 보기전 까지는 CountMissing 클래스 자체만으로 클래스 사용 용도가 무엇인지 이해하기 어렵다는 단점이 있다.
누가 CounterMissing 객체를 생성하고, 언제 missing 메서드를 호출하는지 모른다.
파이썬에서는 클래스에 `__call__` 이라는 특별한 메서드를 정의해서 이런 상황을 명확하게 할 수 있다


In [20]:
class BetterCountMissing(object):
    def __init__(self):
        self.added = 0
    
    def __call__(self):
        self.added += 1
        return 0

In [21]:
counter = BetterCountMissing()
counter()
assert callable(counter)

In [27]:
counter = BetterCountMissing()
result = defaultdict(counter, current)
for key, amount in increments:
    result[key] += amount
assert counter.added == 2

위의 예제가 다음의 이유들로 CountMissing.missing 예제보다 명확하다.
1. `__call__` 메서드는 함수 인수를 사용하기 적합한 위치에 클래스의 인스턴스를 사요할 수 있다는 사실을 말해준다.
2. 이 코드를 처음 보는 사람을 클래스의 주요 동작을 책임지는 진입점으로 안내하는 역할도 한다.
3. 클래스의 목적이 상태 보존 클로저로 동작하는 것이라는 강력한 힌트를 제공핟나.
4. `__call__` 을 사용할 때 defaultdict는 여전히 무슨 일이 일어나는지 모른다.
   defaultdicr에 필요한 건 기본값 후크용 함수뿐이다.